In [ ]:
%load_ext autoreload 
%autoreload 2
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils.pgtools import PGWrangler
import pandas as pd

pgw = PGWrangler()
conn = pgw.conn

# Helper Functions

In [ ]:
def escape_colon_within_quotes(string):
    """ Helper function that replaces ',' with ' ' if in '"' quotes
    
    :param str string: the string in which to replace the ','
    :returns: string with replaced ','
    :rtype: str
    
    """
    ret_string = ""
    quote = False
    for char in string:
        if char == '"':
            if quote:
                quote=False
            else:
                quote=True
        if char == ',' and quote:
            char = ' '
        ret_string+=char
    return ret_string

In [ ]:
def get_info(start,end):
    """ Selects interesting fields from joint table and parses them to dataframe
    
    :param str start: datetime string for start of selection
    :param str end: datetime string for end of selection
    :returns: dataframe with potentially interesting columns
    :rtype: pd.DataFrame
    
    """
    col_names = [('i_eventnumber','id'),
                 ('cdf_response_typ__incident_type_desc','desc'),
                 ('udt4_idisposition__idi_dispositiontext','disp_text'),
                 ('cdf_response_typ__compliment','compliment'),
                 ('dbo_rfirehouseincident__pun_unitid','punit'),
                 ("dbo_rfirehouseapparatus__pun_unitid",'unit'),
                 ("dbo_rfirehouseapparatus__iiu_tdispatch",'tdispatch'),
                 ("dbo_rfirehouseapparatus__iiu_tenroute",'tenroute'),
                 ("dbo_rfirehouseapparatus__iiu_tarrive",'tarrive'),
                 ("dbo_rfirehouseapparatus__iiu_tenroutehospital",'tenroute_h'),
                 ("dbo_rfirehouseapparatus__iiu_tclear","tclear"),
                 ("dbo_rfirehouseincident__v_streetname","street"),
                 ('"t_destination__DESTINNAME"',"hospital"),
                 ("latitude","lat"),
                 ("longitude","long"),
                 ("cdf_response_typ__code","rcode")
                ]
    col_str = ",".join([x for (x,_) in col_names])
    col_names_df = [x for (_,x) in col_names]
    results = conn.execute( 
    """select (%s) 
               from clean.join_2015_10_11_2015_10_14 
               where "i_ttimedispatch" 
               between '%s' 
               and '%s'"""%(col_str,start,end))
    df = pd.DataFrame(sorted([escape_colon_within_quotes(str(x)[3:-4]).split(',') for x in results.fetchall()]))
    df.columns = col_names_df
    return df


In [ ]:
# List of all hospitals locations in data set acquired by searching the hospital in google maps
loc_hospitals = {"Cincinnati Children's Hospital Medical Center":(-84.504674,39.1407851),
                 'The Jewish Hospital':(-84.3836629,39.2058405),
                 'Mercy West (North Bend Rd)':(-84.5973485,39.1906862),
                 'Mercy Hospital Anderson':(-84.351737,39.0857436),
                 'Mercy Hospital Western Hills (Free Standing ER)':(-84.6189937,39.1576889),
                 'Rookwood Medical Center':(-84.4444533,39.1530297),
                 'Veterans Affairs Medical Center':(-84.5094072,39.1394028),
                 'The Christ Hospital':(-84.5127466,39.1212353),
                 'Good Samaritan Hospital':(-84.5236319,39.1397532),
                 'Bethesda North Hospital':(-84.3433869,39.2520961),
                 'University Hospital':(-84.5056878,39.1377437),
                 'Mercy Hospital Fairfield':(-84.5196187,39.3123531)    
}

In [ ]:
def time_location(df,unit):
    """ Helper function that parses the specific df of get_info to a time and location for each unit
    
    :param pd.DataFrame df: Dataframe from get_info
    :param str unit: Identifier for specific transport unit
    :returns: list of times, purposes (p=patient,h=hospital), and locations for given unit
    :rtype: [(str,str,(float,float))]
    
    """
    df_unit = df[df["unit"]==unit]
    times = []
    for (iid,group_df) in df_unit.groupby("id"):
        dispatch_time = list(set(group_df["tdispatch"]))[0]
        clear_time = list(set(group_df["tclear"]))[-1]
        hospital_time = list(set(group_df["tenroute_h"]))[0]
        hospital = sorted(list(set(group_df["hospital"])))[::-1][0].strip('"').replace("\\","")
        latitude = list(set(group_df["lat"]))[0]
        longitude = list(set(group_df["long"]))[0]
        times.append((dispatch_time,'p',(float(longitude),float(latitude))))
        if hospital and hospital_time:
            times.append((hospital_time,'h',loc_hospitals[hospital][::-1]))
        # If the data says that the ambulance went to a hospital, but there is no hospital time, take clear time
        # -> Manually check that clear time is not "too close" to dispatch time
        elif hospital:
            times.append((clear_time,'h_notime',loc_hospitals[hospital][::-1]))
    return times
    

# Data stories

## Story 1: incidents of Unit 23 and Unit 29 on 2015-10-12

In [ ]:
shift_start = '2015-10-12 07:00:00'
shift_end = '2015-10-13 06:59:59'

In [ ]:
df = get_info(shift_start,shift_end)
times_23 = [['M23']+list(x) for x in time_location(df,'M23')]
times_29 = [['M29']+list(x) for x in time_location(df,'M29')]

joint_list = times_23+times_29
joint_list.sort(key=lambda x: x[1])
joint_list  

## Story 2: a specific incident on that day

### 1. Get all incidents during busy period

In [ ]:
start_busy = '2015-10-12 13:10:44' 
end_busy = '2015-10-12 13:41:23'

In [ ]:
get_info(start_busy,end_busy)

### 2. For the selected incident check what the units did before and after

In [ ]:
before = '2015-10-12 12:00:00'
after = '2015-10-12 16:00:00'
df = get_info(before,after)

#### E05

In [ ]:
df[df["unit"]=='E05']

#### T19

In [ ]:
df[df["unit"]=='T19']

#### M17

In [ ]:
df[df["unit"]=='M17']

#### M51

In [ ]:
df[df["unit"]=='M51']